# Creating, Planting, and Finding Signals Within Noise
Elizabeth Kapelevich

In [2]:
import math
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import signals_and_noise as san

In [3]:
Time1, Position1 = np.loadtxt("mock_data_challenge_1.dat").T
Time2, Position2 = np.loadtxt("mock_data_challenge_2.dat").T

## Main Goal for Week of 6/27:
Adjust the `template`, `integrator`, `cross_correlation`, and `fs_search` functions to take a frequency array as input and output the proper result.
#### Tasks:
* $\checkmark$ Within the `template` function, create a meshgrid that will allow many different values of frequencies that can be used by many different values of the `S` equation.
* $\checkmark$ Using a for-loop, evaluate ` S ` over values of ` frequency ` and ` time_stamps ` so that the output is an array of ` S ` values. $\checkmark$
* Once that is complete, go through the ` integrator ` function and the ` cross_correlation ` function and update the code so that the frequency parameter can be properly passed as an array and calculated.
* Note: Adjusting/creating for-loops and meshgrids may help.

#### The main goal is to find the global maximum of the cross-correlation and apply its index to the frequency array to find the corresponding value.

For the following function, the error describes that the index of the `C` array is too large to be applied towards the `frequency` array, therefore the value for `frequency` cannot be found. The goal is to find the largest `C` value's index and apply it to the `frequency` array to find the corresponding value. `C` is a 2-D array composed of a list of integration results at a particular position that are sliding around the data. 

When using `np.argmax()`, it will spit out the index of the largest value in an array. When the array is more than one dimension, it does not specify the row and column of that largest value, it still only outputs its index. To fix this issue, it is necessary to specify the `axis`, in this case `axis = 1` so we are finding the column index. The output will have a list of values that state the amount of elements in an array, which array it is, and the index of the element in that array. The only thing we are looking for is that last value, therefore we add `[-1]` after the `np.argmax()` function.

In [77]:
def frequency_search(del_T_0, t_start, t_max, 
             data_time_series, a, f, sigma, t_duration, del_T):
    
    times, C = san.cross_correlation(del_T_0, t_start, t_max, data_time_series, 
                                 a, f, sigma, t_duration, del_T)
    
    max_index_C = (np.argmax(C,  axis=1)[-1])

    frequency_correct = f[max_index_C]
    
    return frequency_correct

In [78]:
a = 1.5
f = np.array([1, 2, 3])
sigma = 0.1
t_0 = 0
t_start = 2
t_duration = 8
t_max = 20
data_time_series = [Time1, Position1]
data_time_stamps = Time1
del_T_0 = 0.012
del_T = 0.012

In [79]:
search = frequency_search(del_T_0, t_start, t_max, data_time_series, a, f, sigma, t_duration, del_T)

IndexError: index 340 is out of bounds for axis 0 with size 3

In [80]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 9, 8]])
print(np.argmax(a,  axis=1))
print(np.argmax(a,  axis=1)[-1])

[2 2 1]
1
